<a href="https://colab.research.google.com/github/FIFLove/AICC-RAG-evaluation/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#인증서안내_AI_고객센터_챗봇

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

# 데이터 불러오기

In [ ]:
import pandas as pd

In [ ]:
json_file_path = '민원(콜센터) 질의응답_금융보험_잔고 및 거래내역_Training.json'
df = pd.read_json(json_file_path)
df

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
도메인_리스트 = df['도메인'].unique()
for idx, 도메인 in enumerate(도메인_리스트) :
  print(idx, 도메인)

In [ ]:
카테고리_리스트 = df['카테고리'].unique()
for idx, 카테고리 in enumerate(카테고리_리스트) :
  print(idx, 카테고리)

In [ ]:
대화셋일련번호_리스트 = df['대화셋일련번호'].unique()
for idx, 대화셋일련번호 in enumerate(대화셋일련번호_리스트) :
  print(idx, 대화셋일련번호)

In [ ]:
화자_리스트 = df['화자'].unique()
for idx, 화자 in enumerate(화자_리스트) :
  print(idx, 화자)

In [ ]:
문장번호_리스트 = df['문장번호'].unique()
for idx, 문장번호 in enumerate(문장번호_리스트) :
  print(idx, 문장번호)

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트) :
  print(idx, 고객의도)

In [ ]:
상담사의도_리스트 = df['상담사의도'].unique()
for idx, 상담사의도 in enumerate(상담사의도_리스트) :
  print(idx, 상담사의도)

In [ ]:
QA_리스트 = df['QA'].unique()
for idx, QA in enumerate(QA_리스트) :
  print(idx, QA)

In [ ]:
고객질문_요청_리스트 = df['고객질문(요청)'].unique()
for idx, 고객질문_요청 in enumerate(고객질문_요청_리스트) :
  print(idx, 고객질문_요청)

In [ ]:
상담사질문_요청_리스트 = df['상담사질문(요청)'].unique()
for idx, 상담사질문_요청 in enumerate(상담사질문_요청_리스트) :
  print(idx, 상담사질문_요청)

In [ ]:
고객답변_리스트 = df['고객답변'].unique()
for idx, 고객답변 in enumerate(고객답변_리스트) :
  print(idx, 고객답변)

In [ ]:
상담사답변_리스트 = df['상담사답변'].unique()
for idx, 상담사답변 in enumerate(상담사답변_리스트) :
  print(idx, 상담사답변)

In [ ]:
개체명_리스트 = df['개체명 '].unique()
for idx, 개체명 in enumerate(개체명_리스트) :
  print(idx, 개체명)

In [ ]:
용어사전_리스트 = df['용어사전'].unique()
for idx, 용어사전 in enumerate(용어사전_리스트) :
  print(idx, 용어사전)

In [ ]:
지식베이스_리스트 = df['지식베이스'].unique()
for idx, 지식베이스 in enumerate(지식베이스_리스트) :
  print(idx, 지식베이스)

# 고객의도가 '인증서'인 데이터만 필터링하기

In [ ]:
filtered_df = df[df['고객의도']=='인증서']
filtered_df

In [ ]:
len(filtered_df)

# Vectorstore에 저장하기 위한 데이터 정제하기

In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(filtered_df.iterrows()):
  print(count, row['대화셋일련번호'], row['고객질문(요청)'])
  대화셋_set.add(row['대화셋일련번호'])

In [ ]:
대화셋_set

{'A12471', 'A12520', 'A17086', 'A29889', 'A4091', 'A4119', 'A6986'}

In [ ]:
filtered_df = df[df['대화셋일련번호'].isin(대화셋_set)]
filtered_df

In [ ]:
# '대화셋일련번호'로 그룹화하고 각 그룹을 인덱스 순서대로 정렬하여 원하는 컬럼들을 하나의 문자열로 합치기
def combine_columns(group):
  sorted_group = group.sort_index()
  combined_string = " ".join(
      sorted_group[['고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변']].astype(str).values.flatten()
  )
  return combined_string

In [ ]:
combined_result = filtered_df.groupby('대화셋일련번호').apply(combine_columns).reset_index(name='combined_string')
combined_result

<ipython-input-12-fdd242de906f>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_result = filtered_df.groupby('대화셋일련번호').apply(combine_columns).reset_index(name='combined_string')


,대화셋일련번호,combined_string
0,A12471,인증서 연장을 어떻게 하죠? 공인인증서 연장신청에 관한 문의를 주셨군요 ...
1,A12520,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...
2,A17086,인증서 파일로 첨부할수있나요 컴퓨터앞에있는데 자세히 알려주실까요? ...
3,A29889,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...
4,A4091,집과 직장에서 인터넷 뱅킹을 이용가능하나요? 공인인증서가 있습니까? ...
5,A4119,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...
6,A6986,인증서 연장 신청 하려고 하는데요? 공인인증서 연장신청 말씀하시는거죠? ...


In [ ]:
combined_result[combined_result['대화셋일련번호']=='A12471']['combined_string'].values[0]

'인증서 연장을 어떻게 하죠?       공인인증서 연장신청에 관한 문의를 주셨군요   다음달 10일이 만료일이더라구요   그럼 아직 20여일 남아있으신건가요?     네 그정도 기간이 남았네요   공인인증서는 어디에 보관하고 계십니까?     노트북과 USB에 저장되어있어요     인증서 연장하는 절차를 안내해드리겠습니다.   네 알려주세요     OO은행 홈페이지에 접속하시구요   네 접속되어있어요     화면 오른쪽 상단에 공인인증센터 클릭하시면 됩니다.   네 해당페이지에 있습니다.     공인인증서 갱신 클릭하시면 됩니다. 만료일 30일 전부터 갱신가능 이라고 되어있네요?       고객님 공인인증서 기한이 20일 정도 남아서 갱신가능하십니다. 은행/보험용 공인인증서로 갱신하면 되나요?       네 은행 이체용으로 사용하실경우 가능한 인증서 입니다.   갱신 완료하였습니다. '

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A12471']
temp_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
20872,금융/보험,잔고 및 거래내역,A12471,고객,1,인증서,,Q,인증서 연장을 어떻게 하죠?,,,,연장,연장/갱신,"연장,갱신"
20873,금융/보험,잔고 및 거래내역,A12471,상담사,2,,인증서,A,,,,공인인증서 연장신청에 관한 문의를 주셨군요,공인인증서,공인인증서/범용공인인증서,"공인인증서,범용공인인증서"
20874,금융/보험,잔고 및 거래내역,A12471,고객,3,인증서,,A,,,다음달 10일이 만료일이더라구요,,만료,만료/완료,"만료,완료"
20875,금융/보험,잔고 및 거래내역,A12471,상담사,4,,인증서,Q,,그럼 아직 20여일 남아있으신건가요?,,,20여일,20여일/기간,"20여일,기간"
20876,금융/보험,잔고 및 거래내역,A12471,고객,5,인증서,,A,,,네 그정도 기간이 남았네요,,기간,기간/시간,"기간,시간"
20877,금융/보험,잔고 및 거래내역,A12471,상담사,6,,인증서,Q,,공인인증서는 어디에 보관하고 계십니까?,,,보관,보관/저장,"보관,저장"
20878,금융/보험,잔고 및 거래내역,A12471,고객,7,인증서,,A,,,노트북과 USB에 저장되어있어요,,USB,USB/이동형데이타기억장치,"USB,이동형데이타기억장치"
20879,금융/보험,잔고 및 거래내역,A12471,상담사,8,,인증서,A,,,,인증서 연장하는 절차를 안내해드리겠습니다.,절차,절차/순차,"절차,순차"
20880,금융/보험,잔고 및 거래내역,A12471,고객,9,인증서,,A,,,네 알려주세요,,,,
20881,금융/보험,잔고 및 거래내역,A12471,상담사,10,,인증서,A,,,,OO은행 홈페이지에 접속하시구요,접속,접속/연결,"접속,연결"


In [ ]:
# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():
  print(f"대화셋일련번호: {row['대화셋일련번호']}, Combined String: {row['combined_string']}")

대화셋일련번호: A12471, Combined String: 인증서 연장을 어떻게 하죠?       공인인증서 연장신청에 관한 문의를 주셨군요   다음달 10일이 만료일이더라구요   그럼 아직 20여일 남아있으신건가요?     네 그정도 기간이 남았네요   공인인증서는 어디에 보관하고 계십니까?     노트북과 USB에 저장되어있어요     인증서 연장하는 절차를 안내해드리겠습니다.   네 알려주세요     OO은행 홈페이지에 접속하시구요   네 접속되어있어요     화면 오른쪽 상단에 공인인증센터 클릭하시면 됩니다.   네 해당페이지에 있습니다.     공인인증서 갱신 클릭하시면 됩니다. 만료일 30일 전부터 갱신가능 이라고 되어있네요?       고객님 공인인증서 기한이 20일 정도 남아서 갱신가능하십니다. 은행/보험용 공인인증서로 갱신하면 되나요?       네 은행 이체용으로 사용하실경우 가능한 인증서 입니다.   갱신 완료하였습니다. 
대화셋일련번호: A12520, Combined String: 스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. 왜 그러는 거에요?       스마트폰의 메모리가 부족하거나,?공용?WIFI를 사용하는 경우 공인인증서 로그인 후 앱의 구동이 멈추고?다시 초기화면으로 돌아가는 현상이 일부 발생할 수 있습니다.  설치되었던 인터넷뱅킹 앱 삭제 후 최신버전으로 업데이트 하거나?LTE망에 접속하여? 이용하시길 바랍니다 아~내 거 인터넷뱅킹 앱을 업데이트를 안해도 그럴 수 있다는 거네요?       네. 그렇습니다. 고객님   네. 알았어요.  그리고 무료 공인인증서를 증권사에서 사용할 수 있어요?       아닙니다. ?은행에서 발급받은 무료 공인인증서는 증권사 이용이 불가능 합니다. 증권사에서 이용을 원하시는 경우 증권사에서 발급하는?「증권거래용 인증서」를 발급받거나,?은행과 증권사 모두 이용이 가능한 범용공인인증서(한국정보인증)를 발급받으시면 됩니다. 아. 그렇구나. 그럼 증권사에서 인증서를 받

# 중복데이터 제거

- A12520 == A29889 == A4119

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A12520']
temp_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
21034,금융/보험,잔고 및 거래내역,A12520,고객,1,인증서,,Q,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...,,,,스마트뱅킹/공인인증서/비밀번호/초기화면,스마트뱅킹/공인인증서/비밀번호/입력/초기화면,"공인인증서,초기화면"
21035,금융/보험,잔고 및 거래내역,A12520,상담사,2,,인증서,A,,,,"스마트폰의 메모리가 부족하거나,?공용?WIFI를 사용하는 경우 공인인증서 로그인 후...",스마트폰/메모리/공인인증서/초기화면/인터넷뱅킹 앱/LTE망/접속/고객센터 문의,스마트폰/메모리부족/공용 WIFI/공인인증서/로그인/앱/구동/초기화면/설치/인터넷뱅...,"메모리,고객센터문의"
21036,금융/보험,잔고 및 거래내역,A12520,고객,3,인증서,,Q,아~내 거 인터넷뱅킹 앱을 업데이트를 안해도 그럴 수 있다는 거네요?,,,,인터넷뱅킹 앱/업데이트,인터넷뱅킹 앱/업데이트,업데이트
21037,금융/보험,잔고 및 거래내역,A12520,상담사,4,,인증서,A,,,,네. 그렇습니다. 고객님,,,
21038,금융/보험,잔고 및 거래내역,A12520,고객,5,,,A,,,네. 알았어요.,,,,
21039,금융/보험,잔고 및 거래내역,A12520,고객,6,인증서,,Q,그리고 무료 공인인증서를 증권사에서 사용할 수 있어요?,,,,무료 공인인증서/증권사/사용,무료공인인증서/증권사/사용,"증권사,사용"
21040,금융/보험,잔고 및 거래내역,A12520,상담사,7,,인증서,A,,,,아닙니다. ?은행에서 발급받은 무료 공인인증서는 증권사 이용이 불가능 합니다. 증권...,무료공인인증서/증권사/증권거래용 인증서/범용공인인증서,은행/발급/무료 공인인증서/증권사/불가능/증권거래용 인증서/범용공인인증서/한국정보인증,"증권사,한국정보인증"
21041,금융/보험,잔고 및 거래내역,A12520,고객,8,인증서,,Q,아. 그렇구나. 그럼 증권사에서 인증서를 받거나 범용공인인증서를 발급받아야 된다는 거죠?,,,,증권사/인증서/범용공인인증서,증권사/인증서/범용공인인증서/발급,"인증서,발급"
21042,금융/보험,잔고 및 거래내역,A12520,상담사,9,,인증서,A,,,,네. 그렇습니다. 고객님,,,
21043,금융/보험,잔고 및 거래내역,A12520,상담사,10,,인증서,Q,,혹시 다른 문의사항 있으십니까?,,,문의사항,다른 문의사항,"문의사항,다른문의사항"


In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A29889']
temp_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
62036,금융/보험,잔고 및 거래내역,A29889,고객,1,인증서,,Q,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...,,,,스마트뱅킹/공인인증서/비밀번호/초기화면,"스마트뱅킹/모바일뱅킹, 비밀번호/암호","공인인증서,암호"
62037,금융/보험,잔고 및 거래내역,A29889,상담사,2,,인증서,A,,,,"스마트폰의 메모리가 부족하거나,?공용?WIFI를 사용하는 경우 공인인증서 로그인 후...",스마트폰/메모리/공인인증서/초기화면/인터넷뱅킹 앱/LTE망/접속/고객센터 문의,"스마트폰/핸드폰, 인터넷뱅킹/온라인뱅킹, 앱/어플, 고객센터/기관, 문의/질문","메모리,질문"
62038,금융/보험,잔고 및 거래내역,A29889,고객,3,인증서,,Q,아~내 거 인터넷뱅킹 앱을 업데이트를 안해도 그럴 수 있다는 거네요?,,,,인터넷뱅킹 앱/업데이트,"인터넷뱅킹/온라인뱅킹, 앱/어플","업데이트,어플"
62039,금융/보험,잔고 및 거래내역,A29889,상담사,4,,인증서,A,,,,네. 그렇습니다. 고객님,고객님,고객님/손님,"고객님,손님"
62040,금융/보험,잔고 및 거래내역,A29889,고객,5,,A,A,,,네. 알았어요.,,,,
62041,금융/보험,잔고 및 거래내역,A29889,고객,6,인증서,,Q,그리고 무료 공인인증서를 증권사에서 사용할 수 있어요?,,,,무료 공인인증서/증권사/사용,"증권사/증권회사, 사용/이용","증권사,이용"
62042,금융/보험,잔고 및 거래내역,A29889,상담사,7,,인증서,A,,,,아닙니다. ?은행에서 발급받은 무료 공인인증서는 증권사 이용이 불가능 합니다. 증권...,은행/무료공인인증서/증권사/증권거래용 인증서/범용공인인증서,"은행/금융기관, 증권사/증권회사, 인증서/서류","무료공인인증서,서류"
62043,금융/보험,잔고 및 거래내역,A29889,고객,8,인증서,,Q,아. 그렇구나. 그럼 증권사에서 인증서를 받거나 범용공인인증서를 발급받아야 된다는 거죠?,,,,증권사/인증서/범용공인인증서,"증권사/증권회사, 인증서/서류","인증서,서류"
62044,금융/보험,잔고 및 거래내역,A29889,상담사,9,,인증서,A,,,,네. 그렇습니다. 고객님,고객님,고객님/손님,"고객님,손님"
62045,금융/보험,잔고 및 거래내역,A29889,상담사,10,,인증서,Q,,혹시 다른 문의사항 있으십니까?,,,문의사항,문의사항/질문사항,"문의사항,질문사항"


In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A4119']
temp_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
40,금융/보험,잔고 및 거래내역,A4119,고객,1,인증서,,Q,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...,,,,"스마트뱅킹,공인인증서,비밀번호,초기화면",스마트뱅킹/공인인증서/비밀번호/입력/초기화면,"공인인증서,초기화면"
41,금융/보험,잔고 및 거래내역,A4119,상담사,2,,인증서,A,,,,"스마트폰의 메모리가 부족하거나,?공용?WIFI를 사용하는 경우 공인인증서 로그인 후...","스마트폰,메모리,공인인증서,초기화면,인터넷뱅킹 앱,LTE망,접속,고객센터 문의",스마트폰/메모리부족/공용 WIFI/공인인증서/로그인/앱/구동/초기화면/설치/인터넷뱅...,"메모리,고객센터문의"
42,금융/보험,잔고 및 거래내역,A4119,고객,3,인증서,,Q,아~내 거 인터넷뱅킹 앱을 업데이트를 안해도 그럴 수 있다는 거네요?,,,,"인터넷뱅킹 앱,업데이트",인터넷뱅킹 앱/업데이트,업데이트
43,금융/보험,잔고 및 거래내역,A4119,상담사,4,,인증서,A,,,,네. 그렇습니다. 고객님,,,
44,금융/보험,잔고 및 거래내역,A4119,고객,5,인증서,,A,,,네. 알았어요.,,,,
45,금융/보험,잔고 및 거래내역,A4119,고객,6,인증서,,Q,그리고 무료 공인인증서를 증권사에서 사용할 수 있어요?,,,,"무료 공인인증서,증권사,사용",무료공인인증서/증권사/사용,"증권사,사용"
46,금융/보험,잔고 및 거래내역,A4119,상담사,7,,인증서,A,,,,아닙니다. ?은행에서 발급받은 무료 공인인증서는 증권사 이용이 불가능 합니다. 증권...,"무료공인인증서,증권사,증권거래용 인증서,범용공인인증서",은행/발급/무료 공인인증서/증권사/불가능/증권거래용 인증서/범용공인인증서/한국정보인증,"증권사,한국정보인증"
47,금융/보험,잔고 및 거래내역,A4119,고객,8,인증서,,Q,아. 그렇구나. 그럼 증권사에서 인증서를 받거나 범용공인인증서를 발급받아야 된다는 거죠?,,,,"증권사,인증서,범용공인인증서",증권사/인증서/범용공인인증서/발급,"인증서,발급"
48,금융/보험,잔고 및 거래내역,A4119,상담사,9,,인증서,A,,,,네. 그렇습니다. 고객님,,,
49,금융/보험,잔고 및 거래내역,A4119,상담사,10,,인증서,Q,,혹시 다른 문의사항 있으십니까?,,,문의사항,다른 문의사항,"문의사항,다른문의사항"


In [ ]:
combined_result

,대화셋일련번호,combined_string
0,A12471,인증서 연장을 어떻게 하죠? 공인인증서 연장신청에 관한 문의를 주셨군요 ...
1,A12520,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...
2,A17086,인증서 파일로 첨부할수있나요 컴퓨터앞에있는데 자세히 알려주실까요? ...
3,A29889,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...
4,A4091,집과 직장에서 인터넷 뱅킹을 이용가능하나요? 공인인증서가 있습니까? ...
5,A4119,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...
6,A6986,인증서 연장 신청 하려고 하는데요? 공인인증서 연장신청 말씀하시는거죠? ...


In [ ]:
# '대화셋일련번호'가 'A29889', 'A4119'인 행 삭제
combined_result = combined_result[combined_result['대화셋일련번호'] != 'A29889']
combined_result = combined_result[combined_result['대화셋일련번호'] != 'A4119']
combined_result

,대화셋일련번호,combined_string
0,A12471,인증서 연장을 어떻게 하죠? 공인인증서 연장신청에 관한 문의를 주셨군요 ...
1,A12520,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...
2,A17086,인증서 파일로 첨부할수있나요 컴퓨터앞에있는데 자세히 알려주실까요? ...
4,A4091,집과 직장에서 인터넷 뱅킹을 이용가능하나요? 공인인증서가 있습니까? ...
6,A6986,인증서 연장 신청 하려고 하는데요? 공인인증서 연장신청 말씀하시는거죠? ...


In [ ]:
# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():
  print(f"대화셋일련번호: {row['대화셋일련번호']}, Combined String: {row['combined_string']}")

대화셋일련번호: A12471, Combined String: 인증서 연장을 어떻게 하죠?       공인인증서 연장신청에 관한 문의를 주셨군요   다음달 10일이 만료일이더라구요   그럼 아직 20여일 남아있으신건가요?     네 그정도 기간이 남았네요   공인인증서는 어디에 보관하고 계십니까?     노트북과 USB에 저장되어있어요     인증서 연장하는 절차를 안내해드리겠습니다.   네 알려주세요     OO은행 홈페이지에 접속하시구요   네 접속되어있어요     화면 오른쪽 상단에 공인인증센터 클릭하시면 됩니다.   네 해당페이지에 있습니다.     공인인증서 갱신 클릭하시면 됩니다. 만료일 30일 전부터 갱신가능 이라고 되어있네요?       고객님 공인인증서 기한이 20일 정도 남아서 갱신가능하십니다. 은행/보험용 공인인증서로 갱신하면 되나요?       네 은행 이체용으로 사용하실경우 가능한 인증서 입니다.   갱신 완료하였습니다. 
대화셋일련번호: A12520, Combined String: 스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. 왜 그러는 거에요?       스마트폰의 메모리가 부족하거나,?공용?WIFI를 사용하는 경우 공인인증서 로그인 후 앱의 구동이 멈추고?다시 초기화면으로 돌아가는 현상이 일부 발생할 수 있습니다.  설치되었던 인터넷뱅킹 앱 삭제 후 최신버전으로 업데이트 하거나?LTE망에 접속하여? 이용하시길 바랍니다 아~내 거 인터넷뱅킹 앱을 업데이트를 안해도 그럴 수 있다는 거네요?       네. 그렇습니다. 고객님   네. 알았어요.  그리고 무료 공인인증서를 증권사에서 사용할 수 있어요?       아닙니다. ?은행에서 발급받은 무료 공인인증서는 증권사 이용이 불가능 합니다. 증권사에서 이용을 원하시는 경우 증권사에서 발급하는?「증권거래용 인증서」를 발급받거나,?은행과 증권사 모두 이용이 가능한 범용공인인증서(한국정보인증)를 발급받으시면 됩니다. 아. 그렇구나. 그럼 증권사에서 인증서를 받

# Documnet 생성 및 Vectorstore에 Embedding해서 저장

In [ ]:
from  langchain.schema import Document

docs = []

# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():

  doc = Document(page_content=row['combined_string'], metadata={'대화셋일련번호': row['대화셋일련번호']})
  docs.append(doc)

In [ ]:
docs

[Document(metadata={'대화셋일련번호': 'A12471'}, page_content='인증서 연장을 어떻게 하죠?       공인인증서 연장신청에 관한 문의를 주셨군요   다음달 10일이 만료일이더라구요   그럼 아직 20여일 남아있으신건가요?     네 그정도 기간이 남았네요   공인인증서는 어디에 보관하고 계십니까?     노트북과 USB에 저장되어있어요     인증서 연장하는 절차를 안내해드리겠습니다.   네 알려주세요     OO은행 홈페이지에 접속하시구요   네 접속되어있어요     화면 오른쪽 상단에 공인인증센터 클릭하시면 됩니다.   네 해당페이지에 있습니다.     공인인증서 갱신 클릭하시면 됩니다. 만료일 30일 전부터 갱신가능 이라고 되어있네요?       고객님 공인인증서 기한이 20일 정도 남아서 갱신가능하십니다. 은행/보험용 공인인증서로 갱신하면 되나요?       네 은행 이체용으로 사용하실경우 가능한 인증서 입니다.   갱신 완료하였습니다. '),
 Document(metadata={'대화셋일련번호': 'A12520'}, page_content='스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. 왜 그러는 거에요?       스마트폰의 메모리가 부족하거나,?공용?WIFI를 사용하는 경우 공인인증서 로그인 후 앱의 구동이 멈추고?다시 초기화면으로 돌아가는 현상이 일부 발생할 수 있습니다.  설치되었던 인터넷뱅킹 앱 삭제 후 최신버전으로 업데이트 하거나?LTE망에 접속하여? 이용하시길 바랍니다 아~내 거 인터넷뱅킹 앱을 업데이트를 안해도 그럴 수 있다는 거네요?       네. 그렇습니다. 고객님   네. 알았어요.  그리고 무료 공인인증서를 증권사에서 사용할 수 있어요?       아닙니다. ?은행에서 발급받은 무료 공인인증서는 증권사 이용이 불가능 합니다. 증권사에서 이용을 원하시는 경우 증권사에서 발급하는?「증권거래용 인증서」를 발급받거나,?은행과 증권사 모두 이용이 가능한 범용공인인증

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents = docs, embedding=embedding_model)

# Sanity Check

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retrievered_docs = retriever.invoke(
    "인증서 연장을 어떻게 하죠?"
)
print(retrievered_docs[0].page_content)

인증서 연장을 어떻게 하죠?       공인인증서 연장신청에 관한 문의를 주셨군요   다음달 10일이 만료일이더라구요   그럼 아직 20여일 남아있으신건가요?     네 그정도 기간이 남았네요   공인인증서는 어디에 보관하고 계십니까?     노트북과 USB에 저장되어있어요     인증서 연장하는 절차를 안내해드리겠습니다.   네 알려주세요     OO은행 홈페이지에 접속하시구요   네 접속되어있어요     화면 오른쪽 상단에 공인인증센터 클릭하시면 됩니다.   네 해당페이지에 있습니다.     공인인증서 갱신 클릭하시면 됩니다. 만료일 30일 전부터 갱신가능 이라고 되어있네요?       고객님 공인인증서 기한이 20일 정도 남아서 갱신가능하십니다. 은행/보험용 공인인증서로 갱신하면 되나요?       네 은행 이체용으로 사용하실경우 가능한 인증서 입니다.   갱신 완료하였습니다. 


In [ ]:
retrievered_docs = retriever.invoke(
    "인증서 연장 신청 하려고 하는데요?"
)
print(retrievered_docs[0].page_content)

인증서 연장 신청 하려고 하는데요?     공인인증서 연장신청 말씀하시는거죠?     네 만료일이 이달 20일이더라구요   그럼 10여일 남아있으신건가요?     네 그렇네요   공인인증서는 어디에 갖고계신가요?     노트북에 있습니다     공인인증서 연장 절차 안내해드리겠습니다   감사합니다.   홈페이지에 지금 접속 가능하신가요?     접속했습니다.     그럼, 화면 오른쪽 상단에 공인인증센터 클릭하시면 됩니다.   네 클릭했습니다.     공인인증서 갱신 클릭하시면됩니다. 만료일 30일 전부터 갱신이라고 되어있는데요?       네 그래서 고객님은 지금 갱신가능하신겁니다 은행/보험용 공인인증서로 갱신하면 되나요?       네 맞습니다   감사합니다 


# LLM과 연동

##OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "your_OpenAI_key"

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

# 대화 history 저장을 위한 ConversationBufferMemory 설정

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

<ipython-input-34-61e7e3748c65>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [ ]:
memory.load_memory_variables({})

{'history': ''}

In [ ]:
def load_memory(input):
  return memory.load_memory_variables({})["history"]

# 최종 Prompt 및 Chain 생성

In [ ]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 채팅히스토리를 사용하여 마지막 질문에 대답하십시오.
맥락: {context}
채팅히스토리 : {history}
질문: {question}
도움이 되는 답변:"""
rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
# RAG chain
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder

rag_chain = {"context": retriever, "question": RunnablePassthrough(), "history": load_memory} | rag_prompt_custom | llm

# Tracking을 위한 LangSmith설정(Optional)

In [ ]:
import os
from uuid import uuid4
unique_id = uuid4().hex[0:0]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "your_LangSmith_key"

In [ ]:
memory.load_memory_variables({})["history"]

''

In [ ]:
def get_result_and_save_memory(user_question):
  chat_result = rag_chain.invoke(user_question)

  # memory에 저장
  memory.chat_memory.add_user_message(user_question)
  memory.chat_memory.add_ai_message(chat_result.content)

  return chat_result.content

# 인증서 안내 AI 고객센터 챗봇 성능 evaluation (정성적 평가)

In [ ]:
filtered_df = df[(df['고객의도'] == '인증서')]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,금융/보험,잔고 및 거래내역,A4091,고객,1,인증서,,Q,집과 직장에서 인터넷 뱅킹을 이용가능하나요?,,,,"가정, 직장, 인터넷 뱅킹",인터넷뱅킹/금융서비스,"직장,금융서비스"
2,금융/보험,잔고 및 거래내역,A4091,고객,3,인증서,,A,,,네,,,,
4,금융/보험,잔고 및 거래내역,A4091,고객,5,인증서,,Q,인증서 복사는 어떻게 하죠?,,,,"복사, 방법",,방법
6,금융/보험,잔고 및 거래내역,A4091,고객,7,인증서,,Q,이동식 USB말고 다른 방법있나요?,,,,"이동식USB, 방법",이동식 USB/이동식저장기기,"방법,이동식저장기기"
8,금융/보험,잔고 및 거래내역,A4091,고객,9,인증서,,A,,,네,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62046,금융/보험,잔고 및 거래내역,A29889,고객,11,인증서,,A,,,네.,,,,
62048,금융/보험,잔고 및 거래내역,A29889,고객,13,인증서,,Q,브라우저인증이라고 있던데 그게 어떤 거에요?,,,,브라우저인증,,브라우저인증
62050,금융/보험,잔고 및 거래내역,A29889,고객,15,인증서,,Q,그게 무슨 소리에요?,,,,무슨 소리,,무슨소리
62052,금융/보험,잔고 및 거래내역,A29889,고객,17,인증서,,A,,,아. 그렇구나,,,,


In [ ]:
llm_response = get_result_and_save_memory("집과 직장에서 인터넷 뱅킹을 이용가능하나요?")
llm_response

'네, 집과 직장에서 인터넷 뱅킹을 이용할 수 있습니다. 공인인증서가 있다면, 이를 복사하여 사용할 수 있습니다. 공인인증서를 하드디스크에서 이동식 USB로 복사하거나, 기기간에 복사할 수 있는 방법도 있습니다. 또한, 이메일을 통해 인증서를 전송하여 사용할 수도 있습니다. 노트북에서도 동일한 방법으로 인터넷 뱅킹을 이용할 수 있습니다.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 집과 직장에서 인터넷 뱅킹을 이용가능하나요?\nAI: 네, 집과 직장에서 인터넷 뱅킹을 이용할 수 있습니다. 공인인증서가 있다면, 이를 복사하여 사용할 수 있습니다. 공인인증서를 하드디스크에서 이동식 USB로 복사하거나, 기기간에 복사할 수 있는 방법도 있습니다. 또한, 이메일을 통해 인증서를 전송하여 사용할 수도 있습니다. 노트북에서도 동일한 방법으로 인터넷 뱅킹을 이용할 수 있습니다.'

In [ ]:
# 위와 같은 방식을 사용해서 질문 가능

In [ ]:
# 대화내역을 저장한 Memory 초기화하기
# memory.clear()